In [ ]:
import os

import numpy as np
import xarray as xr
import pandas as pd

from mne import EpochsArray, create_info

from frites.dataset import DatasetEphy

import matplotlib.pyplot as plt

---
# **0 - Functions**
## Load the data of a single subject

In [ ]:
def load_ss(root, subject_nb):
    """Load the data of a single subject.
    
    Parameters
    ----------
    root : string
        Root path where the dataset is stored
    subject_nb : int
        Subject number [0, 12]
    
    Returns
    -------
    hga : xarray.DataArray
        Xarray containing the high-gamma activity
    anat : pandas.DataFrame
        Table containing the anatomical informations
    beh : pandas.DataFrame
        Table containing the behavioral informations
    """
    print(f"Loading the data of subject #{subject_nb}")

    # load the high-gamma activity
    file_hga = os.path.join(root, 'hga', f'hga_s-{subject_nb}.nc')
    hga = xr.load_dataarray(file_hga)

    # load the name of the brain regions
    file_anat = os.path.join(root, 'anat', f'anat_s-{subject_nb}.xlsx')
    anat = pd.read_excel(file_anat)

    # load the behavior
    file_beh = os.path.join(root, 'beh', f'beh_s-{subject_nb}.xlsx')
    beh = pd.read_excel(file_beh)
    
    return hga, anat, beh


---

# **1. Load fresh data**

In [ ]:
###############################################################################
root = '/home/etienne/DATA/Study/PBLT/gdr/'
###############################################################################

# load the data of subject #0
subject_nb = 0
hga, anat, beh = load_ss(root, subject_nb)

---
# **2. `DatasetEphy` with the Prediction Error**
## 2.1 Plot the Prediction error

In [ ]:
# get the prediction error
pe = beh['PE']

# get the block
block = beh['block']

# select pe for block 1
pe_b1 = pe[block == 1]

# plot the PE
plt.plot(pe_b1)
plt.xlabel("Number of trials")
plt.ylabel('Prediction Error')

"""
Pro tip #1
-----------
You can use directly pandas for selecting it
""";
# beh.set_index('block').loc[1, 'PE']

"""
Pro tip #2
----------
if you have seaborn (or plotly) installed, you can color by block !
""";
# plot using seaborn (if installed)
# import seaborn as sns
# sns.lineplot(data=beh, y='PE', hue='block', x=np.arange(len(beh)))

# plot using plotly (if installed)
# import plotly.express as px
# px.line(beh, y='PE', color='block', hover_data=['valence'])

## 2.2 Set the Prediction error as the trial coordinate

In [ ]:
# rename the 'trial' dimension to be 'pe'
hga = hga.rename(trials='pe')

# fill the dimension 'pe' with the values of the PE
hga['pe'] = list(beh['PE'])

## 2.3 Define a `DatasetEphy` for a single subject

In [ ]:
ds = DatasetEphy([hga], y='pe', roi='channels', times='times')
ds

# **3. `DatasetEphy` with brain regions**
## 3.1 Drop channels - replace with brain region names

In [ ]:
# get brain region names
parcels = list(anat['roi'])

# rename the dimension channels to be 'parcels'
hga = hga.rename(channels='parcels')

# fill this dimension with brain regino names
hga['parcels'] = parcels

# create the DatasetEphy
ds = DatasetEphy([hga], y='pe', roi='parcels', times='times')
ds

## 3.2 Do the same, for all of the subjects

In [ ]:
hga = []
for n_s in range(12):
    # load the data of the subject
    _hga, _anat, _beh = load_ss(root, n_s)
    
    # use the PE instead of stimulus
    _hga = _hga.rename(trials='pe')
    _hga['pe'] = list(_beh['PE'])
    
    # use the brain regions instead of channel names
    _hga = _hga.rename(channels='parcels')
    _hga['parcels'] = list(_anat['roi'])
    
    # add this subject to the whole list
    hga.append(_hga)

# finally, build the DatasetEphy
ds = DatasetEphy(hga, y='pe', times='times', roi='parcels')
ds